## NWP vs ML-model Verification

This script verifies output from a ML-based approach versus a traditional NWP
system to model the atmospheric system. The defaults set at the top of this
script are tailored to the Alps-Tödi HPC system at CSCS.

In [100]:
### DEFAULTS ###
PATH_NWP = "/store/ERA5/weatherbench2_original"
PATH_ML = (
    "/capstor/scratch/cscs/flehmann/HydroClim/Models/Aurora/pretrainedAurora_"
    "1460predictions_2021-01-01T00"
)
LEVELS = [150, 500, 850, 1000]
TEMPORAL_RESOLUTION = 6  # in hours
DATETIMES = ["2021-01-01T06", "2021-01-05T00"]
LATITUDES = [90.0, -89.75]  # Aurora has no value at South Pole (90°S = -90°)
VARIABLES_2D = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "geopotential_at_surface",
    "mean_sea_level_pressure",
]
VARIABLES_3D = [
    "geopotential",
    "specific_humidity",
    "temperature",
    "u_component_of_wind",
    "v_component_of_wind",
]

In [101]:
import xarray as xr

Loading ERA5 weatherbench data, originally stored in a Google Cloud Bucket, it
is already available on Alps.

In [105]:
ds = xr.open_zarr("/store/ERA5/weatherbench2_original")
ds = ds.sel(level=LEVELS, latitude=slice(*LATITUDES), time=slice(*DATETIMES))
ds = ds[VARIABLES_2D + VARIABLES_3D]
ds_timestep = int(
    (ds.time.isel(time=1) - ds.time.isel(time=0)).dt.total_seconds() / 3600
)
ds_timestep_factor = TEMPORAL_RESOLUTION // ds_timestep
ds = ds.isel(time=slice(None, None, ds_timestep_factor))
ds

<xarray.Dataset> Size: 2GB
Dimensions:                  (time: 16, latitude: 720, longitude: 1440, level: 4)
Coordinates:
  * latitude                 (latitude) float32 3kB 90.0 89.75 ... -89.5 -89.75
  * level                    (level) int64 32B 150 500 850 1000
  * longitude                (longitude) float32 6kB 0.0 0.25 ... 359.5 359.8
  * time                     (time) datetime64[ns] 128B 2021-01-01T06:00:00 ....
Data variables:
    10m_u_component_of_wind  (time, latitude, longitude) float32 66MB ...
    10m_v_component_of_wind  (time, latitude, longitude) float32 66MB ...
    2m_temperature           (time, latitude, longitude) float32 66MB ...
    geopotential_at_surface  (latitude, longitude) float32 4MB ...
    mean_sea_level_pressure  (time, latitude, longitude) float32 66MB ...
    geopotential             (time, level, latitude, longitude) float32 265MB ...
    specific_humidity        (time, level, latitude, longitude) float32 265MB ...
    temperature              (time, level, latitude, longitude) float32 265MB ...
    u_component_of_wind      (time, level, latitude, longitude) float32 265MB ...
    v_component_of_wind      (time, level, latitude, longitude) float32 265MB ...

In [106]:
ds_ml = xr.open_zarr(
    "/capstor/scratch/cscs/flehmann/HydroClim/Models/Aurora/pretrainedAurora_"
    "1460predictions_2021-01-01T00"
)
ds_ml = ds_ml.sel(level=LEVELS, latitude=slice(*LATITUDES), time=slice(*DATETIMES))
ds_ml = ds_ml[VARIABLES_2D + VARIABLES_3D]
ds_ml_timestep = int(
    (ds.time.isel(time=1) - ds.time.isel(time=0)).dt.total_seconds() / 3600
)
ds_ml_timestep_factor = TEMPORAL_RESOLUTION // ds_ml_timestep
ds_ml = ds_ml.isel(time=slice(None, None, ds_ml_timestep_factor))
ds_ml

<xarray.Dataset> Size: 2GB
Dimensions:                  (time: 16, latitude: 720, longitude: 1440, level: 4)
Coordinates:
  * latitude                 (latitude) float32 3kB 90.0 89.75 ... -89.5 -89.75
  * level                    (level) int64 32B 150 500 850 1000
  * longitude                (longitude) float32 6kB 0.0 0.25 ... 359.5 359.8
  * time                     (time) datetime64[ns] 128B 2021-01-01T06:00:00 ....
Data variables:
    10m_u_component_of_wind  (time, latitude, longitude) float32 66MB ...
    10m_v_component_of_wind  (time, latitude, longitude) float32 66MB ...
    2m_temperature           (time, latitude, longitude) float32 66MB ...
    geopotential_at_surface  (latitude, longitude) float32 4MB ...
    mean_sea_level_pressure  (time, latitude, longitude) float32 66MB ...
    geopotential             (time, level, latitude, longitude) float32 265MB ...
    specific_humidity        (time, level, latitude, longitude) float32 265MB ...
    temperature              (time, level, latitude, longitude) float32 265MB ...
    u_component_of_wind      (time, level, latitude, longitude) float32 265MB ...
    v_component_of_wind      (time, level, latitude, longitude) float32 265MB ...
Attributes:
    description:  pretrained Aurora prediction from 2021-01-01T00
    it0:          90584

In [107]:
assert ds.sizes == ds_ml.sizes, (
    "Dimensions do not match. Please make sure that both datasets contain the\n"
    "exact same dimensions with the same names, the same order and the same length."
    " Currently the sizes are: \n"
    f"ERA5: {ds.sizes}\n"
    f"Aurora: {ds_ml.sizes}"
)

In [108]:
assert ds.data_vars.keys() == ds_ml.data_vars.keys(), (
    "Variables do not match. Please make sure that both datasets contain the\n"
    "exact same variables with the same names. Currently the variables are: \n"
    f"ERA5: {ds.data_vars.keys()}\n"
    f"Aurora: {ds_ml.data_vars.keys()}"
)